In [1]:
import numpy as np
import random as rn
import findspark
import sys
findspark.init("/opt/spark")

In [2]:
from pyspark import SparkContext
sc = SparkContext("local","PSO Distribuido")

In [3]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred[i])**2

    resultado = resultado / n

    return resultado

In [4]:
V_max= 5.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo =[50,50,50]
ind=0
filas_x=[0,0,0]

In [21]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 1000000
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(best_global_fitness)
mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [6]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        posiciones_.append(posicion)
        posiciones= np.array(posiciones_)
        mejor_posiciones_locales_=posiciones_
        mejor_posiciones_locales=np.array(mejor_posiciones_locales_)
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        velocidades_.append(velocidad)
        velocidades= np.array(velocidades_)
        fit = MSE(posiciones[j],objetivo)
        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = mejor_posiciones_locales[j][:]
    part= np.array([posiciones,velocidades,mejor_posiciones_locales])
    return blf,bgf,mejor_pos_global,part 

In [7]:
def fitnessEval(part,mpg,blf,bgf, ind):
    filas=part[0:1,ind]
    fit = MSE(filas[0],objetivo)
    if fit< blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
        part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    return part,mpg,blf,bgf

In [8]:
#ind=sc.accumulator(0)
def fitnessEval_x(part,mpg,blf,bgf):
    global ind
    filas=part[0:1]
    fit = MSE(filas[0],objetivo)
    if fit < blf[ind]:
        blf[ind] = fit
        #mpl[ind] = part[0:1,ind]
      #  part[2:,ind]=filas
        if fit < bgf:
            bgf = fit
            mpg = filas[0]
            accum= fit
    if ind<10:
        ind+=1
    else:
        ind=0
    return part,mpg,blf,bgf

In [9]:
#def posEval(part,mpl,mpg,ind):
def posEval(part,mpg,ind,N):
    velocidades=part[1:2,ind][0]
    mpl=part[2:,ind][0]
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1,ind][0][k]) + c_2*r_2*(mpg[k] - part[0:1,ind][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1,ind][0][k] = part[0:1,ind][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [10]:
#def posEval(part,mpl,mpg,ind):
def posEval_x(part,mpg,N):
    global ind
    velocidades=part[1:2][0]
    mpl=part[2:][0]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[0:1][0][k]) + c_2*r_2*(mpg[k] - part[0:1][0][k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[0:1][0][k] = part[0:1][0][k] + velocidades[k]
    #return part,velocidades
    return part

In [22]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)
#print("particulas[0:1]-> ",particulas[0:1][0])
#print("mejor velocidades-> ",particulas[1:2][0])
#print("mejor posiciones locales-> ",particulas[2:][0])
#print("mejor_pos_global-> ",mejor_pos_global)
#print("best_global_fitness",best_global_fitness)
#print("best_local_fitness",best_local_fitness)

In [23]:
rdd_master= sc.parallelize(particulas)
#print(rdd_master.collect())
for i in range (I):
    resultado = rdd_master.map(lambda x:fitnessEval_x(x,mejor_pos_global,best_local_fitness,best_global_fitness))
    #print(rdd_master.collect())
    resultado_2 = resultado.map(lambda x:posEval_x(x[0],mejor_pos_global,n))
    resultado = resultado_2
#resultado = resultado_2.map(lambda x:fitnessEval_x(x,mejor_pos_global,best_local_fitness,best_global_fitness)).collect()
resultado = resultado.collect()
print(resultado)
particulas = resultado[0]
print("particulas-> ",particulas[0])
print("mejor_pos_global-> ",mejor_pos_global)

[array([[ -64.34680639, -100.42500179,  -42.21046294],
       [   5.        ,   -5.        ,   -5.        ],
       [  93.93535393,  -88.99629085,  -43.80559393],
       [  69.53610996,  -50.76444349,  -74.58198781],
       [ -88.05543561,  -49.965904  ,   38.65387257],
       [ -38.75001531,   32.19831732,  -40.72532277],
       [ -65.19092425,  -58.46169141,   85.16901001],
       [ -17.0189836 ,  -45.30343461,  -90.47298053],
       [ -65.52472582,   13.23670172,   99.40106792],
       [  57.76721411,   29.86386748,  -70.27425077]]), array([[ -4.71224705,  10.53213237,  61.77581238],
       [  5.        ,  -5.        ,  -5.        ],
       [-96.12189039, -13.30681796,  -4.88562223],
       [ 77.70294404,  11.37616203,  98.93994071],
       [-30.89517655, -27.94094467,   9.2704647 ],
       [-54.91450986, -67.96836136, -89.54454112],
       [ 95.41184899, -14.77759563,  44.02256967],
       [ 76.64479782, -78.47585083, -13.69107069],
       [-77.3694284 ,   3.6775297 ,  82.08696635]